In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense , Flatten ,Embedding,Input,LSTM, Bidirectional, Dense, Dropout,GRU,SpatialDropout1D, Conv1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from wordcloud import WordCloud, STOPWORDS
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

2022-04-04 23:30:34.075475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-04 23:30:34.075560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [2]:
file = 'content_english/english/train_tweets.csv'
data_train = pd.read_csv(file, encoding='latin-1')
file = 'content_english/english/test_tweets.csv'
data_test = pd.read_csv(file, encoding='latin-1')

In [3]:
data_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
data_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [4]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [5]:
from nltk import WordNetLemmatizer
def tokenize(text):
    text = re.split('\s+' ,text)
    return [x.lower() for x in text]
def remove_small_words(text):
    return [x for x in text if len(x) > 2 ]
def remove_stopwords(text):
    return [word for word in text if word not in nltk.corpus.stopwords.words('english')]

def lemmatize(text):
    word_net = WordNetLemmatizer()
    return [word_net.lemmatize(word) for word in text]
def return_sentences(tokens):
    return " ".join([word for word in tokens])

data_train['tokens'] = data_train['tweet'].apply(lambda msg : tokenize(msg))
data_train['tokens'] = data_train['tokens'].apply(lambda x : remove_small_words(x))
data_train['tokens'] = data_train['tokens'].apply(lambda x : remove_stopwords(x))
data_train['tokens'] = data_train['tokens'].apply(lambda x : lemmatize(x))
data_train['clean_text'] = data_train['tokens'].apply(lambda x : return_sentences(x))


data_test['tokens'] = data_test['tweet'].apply(lambda msg : tokenize(msg))
data_test['tokens'] = data_test['tokens'].apply(lambda x : remove_small_words(x))
data_test['tokens'] = data_test['tokens'].apply(lambda x : remove_stopwords(x))
data_test['tokens'] = data_test['tokens'].apply(lambda x : lemmatize(x))
data_test['clean_text'] = data_test['tokens'].apply(lambda x : return_sentences(x))

ERROR: Could not find a version that satisfies the requirement nltk (from versions: none)
ERROR: No matching distribution found for nltk


[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train['clean_text'], data_train['label'],random_state = 0,test_size = 0.2)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((25569,), (6393,), (25569,), (6393,))

In [7]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

MAX_SEQUENCE_LENGTH = 60

from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(X_train),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(X_test),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

Vocabulary Size : 38895
Training X Shape: (25569, 60)
Testing X Shape: (6393, 60)


In [8]:
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)
print(y_train.shape, y_test.shape)

(25569, 1) (6393, 1)


In [15]:
# Model configuration
batch_size = 64
no_classes = 100
no_epochs = 5
optimizer = Adam()
verbosity = 1
num_folds = 10

In [16]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
inputs.shape, targets.shape

((31962, 60), (31962, 1))

In [11]:
from keras.layers import Layer
from keras import backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [42]:
inputs, targets

(array([[    0,     0,     0, ...,     2,   639,  2501],
        [    0,     0,     0, ..., 13533, 13534,  8643],
        [    0,     0,     0, ...,   686,   238,  3529],
        ...,
        [    0,     0,     0, ...,   839,   539,    53],
        [    0,     0,     0, ...,   898,   159,   474],
        [    0,     0,     0, ...,  6020,  9428,    30]], dtype=int32),
 array([[0],
        [0],
        [0],
        ...,
        [0],
        [1],
        [0]]))

In [43]:
Word2vec_train_data = list(map(lambda x: x.split(), X_train))
print(Word2vec_train_data[5])

['trump', 'clinton', 'leading,', 'american', 'choose', 'lesser', 'evil..', 'na-mds', 'bernie', 'sanders..']


In [45]:
EMBEDDING_DIM = 100
from gensim.models import Word2Vec
word2vec_model = Word2Vec(Word2vec_train_data,
                 vector_size=EMBEDDING_DIM,
                 workers=3,
                 min_count=4)

In [46]:
word2vec_model.save('word2vec_model.model')

In [47]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, token in tokenizer.word_index.items():
    if word2vec_model.wv.__contains__(word):
        embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

print("Embedding Matrix Shape:", embedding_matrix.shape)

Embedding Matrix Shape: (38895, 100)


In [50]:
str(1)

'1'

In [52]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    model_path = "content_english/models/" + str(fold_no) + "model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "content_english/History/" + str(fold_no) + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

2022-03-27 03:20:03.767755: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10% of free system memory.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
450/450 [==============================] - 642s 1s/step - loss: 0.2486 - accuracy: 0.9289
Epoch 2/5
450/450 [==============================] - 619s 1s/step - loss: 0.2217 - accuracy: 0.9324
Epoch 3/5
450/450 [==============================] - 602s 1s/step - loss: 0.2110 - accuracy: 0.9347
Epoch 4/5
450/450 [==============================] - 518s 1s/step - loss: 0.2043 - accuracy: 0.9354
Epoch 5/5
450/450 [==============================] - 523s 1s/step - loss: 0.2002 - accuracy: 0.9358


2022-03-27 04:09:06.563425: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-27 04:09:38.003473: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: content_english/models/1model/assets


Score for fold 1: loss of 0.2090567946434021; accuracy of 93.7441349029541%


...

Score for fold 8: loss of 0.19146662950515747; accuracy of 93.83604526519775%
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/5
450/450 [==============================] - 429s 917ms/step - loss: 0.2435 - accuracy: 0.9287
Epoch 2/5
450/450 [==============================] - 414s 919ms/step - loss: 0.2219 - accuracy: 0.9313
Epoch 3/5
450/450 [==============================] - 412s 916ms/step - loss: 0.2136 - accuracy: 0.9336
Epoch 4/5
450/450 [==============================] - 415s 923ms/step - loss: 0.2069 - accuracy: 0.9348
Epoch 5/5
450/450 [==============================] - 417s 927ms/step - loss: 0.2027 - accuracy: 0.9351
INFO:tensorflow:Assets written to: content_english/models/9model/assets


INFO:tensorflow:Assets written to: content_english/models/9model/assets


Score for fold 9: loss of 0.19126370549201965; accuracy of 94.11764740943909%
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/5
450/450 [==============================] - 430s 925ms/step - loss: 0.2467 - accuracy: 0.9301
Epoch 2/5
450/450 [==============================] - 414s 920ms/step - loss: 0.2219 - accuracy: 0.9321
Epoch 3/5
450/450 [==============================] - 420s 934ms/step - loss: 0.2146 - accuracy: 0.9337
Epoch 4/5
450/450 [==============================] - 415s 923ms/step - loss: 0.2056 - accuracy: 0.9351
Epoch 5/5
450/450 [==============================] - 395s 878ms/step - loss: 0.2014 - accuracy: 0.9357
INFO:tensorflow:Assets written to: content_english/models/10model/assets


INFO:tensorflow:Assets written to: content_english/models/10model/assets


Score for fold 10: loss of 0.193878173828125; accuracy of 93.52315664291382%
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.2090567946434021 - Accuracy: 93.7441349029541%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.20801396667957306 - Accuracy: 93.27494502067566%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.2031553089618683 - Accuracy: 93.46057772636414%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.20997881889343262 - Accuracy: 93.27284097671509%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.20757953822612762 - Accuracy: 92.95995235443115%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.183108389377594 - Accuracy: 94.3053841590

In [12]:
fastText_train_data = list(map(lambda x: x.split(), X_train))
print(fastText_train_data[5])
EMBEDDING_DIM = 100
from gensim.models import FastText
fastText_model = FastText(fastText_train_data,
                 vector_size=EMBEDDING_DIM,
                 workers=3,
                 min_count=4)

['trump', 'clinton', 'leading,', 'american', 'choose', 'lesser', 'evil..', 'na-mds', 'bernie', 'sanders..']


/home/mostafa/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [56]:
fastText_model.save('content_english/fastText_model.model')

In [13]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, token in tokenizer.word_index.items():
    if fastText_model.wv.__contains__(word):
        embedding_matrix[token] = fastText_model.wv.__getitem__(word)

print("Embedding Matrix Shape:", embedding_matrix.shape)

Embedding Matrix Shape: (38895, 100)


In [14]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    model_path = "content_english/models_fasttext/" + str(fold_no) + "model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "content_english/History_fasttext/" + str(fold_no) + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
450/450 [==============================] - 498s 1s/step - loss: 0.2216 - accuracy: 0.9289
Epoch 2/5
450/450 [==============================] - 367s 816ms/step - loss: 0.1894 - accuracy: 0.9351
Epoch 3/5
450/450 [==============================] - 368s 818ms/step - loss: 0.1838 - accuracy: 0.9368
Epoch 4/5
450/450 [==============================] - 390s 866ms/step - loss: 0.1780 - accuracy: 0.9378
Epoch 5/5
450/450 [==============================] - 429s 953ms/step - loss: 0.1749 - accuracy: 0.9393
INFO:tensorflow:Assets written to: content_english/models_fasttext/1model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/1model/assets


Score for fold 1: loss of 0.19717951118946075; accuracy of 93.40006113052368%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
450/450 [==============================] - 570s 1s/step - loss: 0.2188 - accuracy: 0.9282
Epoch 2/5
450/450 [==============================] - 417s 927ms/step - loss: 0.1946 - accuracy: 0.9339
Epoch 3/5
450/450 [==============================] - 404s 898ms/step - loss: 0.1847 - accuracy: 0.9363
Epoch 4/5
450/450 [==============================] - 423s 941ms/step - loss: 0.1830 - accuracy: 0.9357
Epoch 5/5
450/450 [==============================] - 418s 929ms/step - loss: 0.1782 - accuracy: 0.9383
INFO:tensorflow:Assets written to: content_english/models_fasttext/2model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/2model/assets


Score for fold 2: loss of 0.16356654465198517; accuracy of 94.55739855766296%
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/5
450/450 [==============================] - 585s 1s/step - loss: 0.2190 - accuracy: 0.9279
Epoch 2/5
450/450 [==============================] - 711s 2s/step - loss: 0.1917 - accuracy: 0.9335
Epoch 3/5
450/450 [==============================] - 534s 1s/step - loss: 0.1854 - accuracy: 0.9369
Epoch 4/5
450/450 [==============================] - 568s 1s/step - loss: 0.1799 - accuracy: 0.9368
Epoch 5/5
450/450 [==============================] - 576s 1s/step - loss: 0.1789 - accuracy: 0.9377
INFO:tensorflow:Assets written to: content_english/models_fasttext/3model/assets


...

Score for fold 8: loss of 0.1791807860136032; accuracy of 93.6170220375061%
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/5
450/450 [==============================] - 460s 988ms/step - loss: 0.2208 - accuracy: 0.9300
Epoch 2/5
450/450 [==============================] - 445s 987ms/step - loss: 0.1913 - accuracy: 0.9343
Epoch 3/5
450/450 [==============================] - 446s 992ms/step - loss: 0.1835 - accuracy: 0.9370
Epoch 4/5
450/450 [==============================] - 444s 985ms/step - loss: 0.1796 - accuracy: 0.9383
Epoch 5/5
450/450 [==============================] - 445s 988ms/step - loss: 0.1769 - accuracy: 0.9392
INFO:tensorflow:Assets written to: content_english/models_fasttext/9model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/9model/assets


Score for fold 9: loss of 0.18089112639427185; accuracy of 93.6170220375061%
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/5
450/450 [==============================] - 460s 990ms/step - loss: 0.2235 - accuracy: 0.9299
Epoch 2/5
450/450 [==============================] - 444s 986ms/step - loss: 0.1912 - accuracy: 0.9341
Epoch 3/5
450/450 [==============================] - 446s 991ms/step - loss: 0.1840 - accuracy: 0.9363
Epoch 4/5
450/450 [==============================] - 444s 986ms/step - loss: 0.1801 - accuracy: 0.9376
Epoch 5/5
450/450 [==============================] - 445s 988ms/step - loss: 0.1776 - accuracy: 0.9385
INFO:tensorflow:Assets written to: content_english/models_fasttext/10model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/10model/assets


Score for fold 10: loss of 0.1745366007089615; accuracy of 94.11764740943909%
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.16640818119049072 - Accuracy: 94.43228244781494%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.19717951118946075 - Accuracy: 93.40006113052368%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.16356654465198517 - Accuracy: 94.55739855766296%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.1657007932662964 - Accuracy: 94.14893388748169%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.18245145678520203 - Accuracy: 93.71088743209839%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.1545303910970688 - Accuracy: 94.680851

In [13]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 38895


In [14]:
embedding_index={}
f = open('content_english/glove.6B.100d.txt',encoding='latin-1')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    embedding_index[word]=coefs
f.close()
tok = Tokenizer()
# create a embeddings matrix with 100 dimenstion
EMBEDDING_DIM=100
embedding_matrix=np.zeros((vocab_size,EMBEDDING_DIM))
for word, i in tok.word_index.items():
    if i>vocab_size:
        continue
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [15]:
print("Embedding Matrix Shape:", embedding_matrix.shape)

Embedding Matrix Shape: (38895, 100)


In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    model_path = "content_english/models_glove/" + str(fold_no) + "model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "content_english/History_glove/" + str(fold_no) + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

2022-03-30 12:28:19.664696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-30 12:28:19.664845: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-30 12:28:19.664929: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mostafa): /proc/driver/nvidia/version does not exist
2022-03-30 12:28:19.745296: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 12:28:21.114905: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
450/450 [==============================] - 405s 834ms/step - loss: 0.6084 - accuracy: 0.9306
Epoch 2/5
450/450 [==============================] - 441s 979ms/step - loss: 0.4782 - accuracy: 0.9306
Epoch 3/5
450/450 [==============================] - 649s 1s/step - loss: 0.3937 - accuracy: 0.9306
Epoch 4/5
450/450 [==============================] - 772s 2s/step - loss: 0.3396 - accuracy: 0.9306
Epoch 5/5
450/450 [==============================] - 711s 2s/step - loss: 0.3051 - accuracy: 0.9306


2022-03-30 13:20:01.327052: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-30 13:20:52.654068: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: content_english/models_glove/1model/assets


Score for fold 1: loss of 0.3053912818431854; accuracy of 92.27400422096252%


2022-03-30 13:21:32.089368: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10% of free system memory.


------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
450/450 [==============================] - 744s 2s/step - loss: 0.6086 - accuracy: 0.9297
Epoch 2/5
450/450 [==============================] - 698s 2s/step - loss: 0.4786 - accuracy: 0.9297
Epoch 3/5
450/450 [==============================] - 696s 2s/step - loss: 0.3945 - accuracy: 0.9297
Epoch 4/5
450/450 [==============================] - 514s 1s/step - loss: 0.3407 - accuracy: 0.9297
Epoch 5/5
450/450 [==============================] - 445s 989ms/step - loss: 0.3065 - accuracy: 0.9297


In [14]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    model_path = "content_english/models_fasttext/" + str(fold_no) + "model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "content_english/History_fasttext/" + str(fold_no) + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

2022-04-04 23:36:08.701038: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-04 23:36:08.701132: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-04 23:36:08.701193: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mostafa): /proc/driver/nvidia/version does not exist
2022-04-04 23:36:08.712085: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
450/450 [==============================] - 189s 398ms/step - loss: 0.2375 - accuracy: 0.9291
Epoch 2/5
450/450 [==============================] - 168s 373ms/step - loss: 0.1987 - accuracy: 0.9317
Epoch 3/5
450/450 [==============================] - 167s 372ms/step - loss: 0.1918 - accuracy: 0.9338
Epoch 4/5
450/450 [==============================] - 167s 371ms/step - loss: 0.1873 - accuracy: 0.9346
Epoch 5/5
450/450 [==============================] - 167s 371ms/step - loss: 0.1836 - accuracy: 0.9362


2022-04-04 23:36:09.086653: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10% of free system memory.
2022-04-04 23:50:49.116375: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-04-04 23:51:08.832321: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15558000 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: content_english/models_fasttext/1model/assets


Score for fold 1: loss of 0.1863562911748886; accuracy of 93.53455901145935%


...

Score for fold 8: loss of 0.18532584607601166; accuracy of 93.68430376052856%
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/5
450/450 [==============================] - 180s 379ms/step - loss: 0.2409 - accuracy: 0.9287
Epoch 2/5
450/450 [==============================] - 172s 382ms/step - loss: 0.2027 - accuracy: 0.9307
Epoch 3/5
450/450 [==============================] - 171s 381ms/step - loss: 0.1927 - accuracy: 0.9330
Epoch 4/5
450/450 [==============================] - 171s 380ms/step - loss: 0.1896 - accuracy: 0.9344
Epoch 5/5
450/450 [==============================] - 171s 380ms/step - loss: 0.1867 - accuracy: 0.9355
INFO:tensorflow:Assets written to: content_english/models_fasttext/9model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/9model/assets


Score for fold 9: loss of 0.1735035479068756; accuracy of 93.9497172832489%
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/5
450/450 [==============================] - 177s 377ms/step - loss: 0.2377 - accuracy: 0.9291
Epoch 2/5
450/450 [==============================] - 167s 371ms/step - loss: 0.2035 - accuracy: 0.9310
Epoch 3/5
450/450 [==============================] - 170s 378ms/step - loss: 0.1914 - accuracy: 0.9335
Epoch 4/5
450/450 [==============================] - 169s 374ms/step - loss: 0.1887 - accuracy: 0.9346
Epoch 5/5
450/450 [==============================] - 170s 377ms/step - loss: 0.1857 - accuracy: 0.9355
INFO:tensorflow:Assets written to: content_english/models_fasttext/10model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/10model/assets


Score for fold 10: loss of 0.16960830986499786; accuracy of 94.09732222557068%
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.1863562911748886 - Accuracy: 93.53455901145935%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.18005912005901337 - Accuracy: 94.35774087905884%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.1827273815870285 - Accuracy: 93.35522651672363%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.1777176856994629 - Accuracy: 93.66029500961304%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.16573284566402435 - Accuracy: 94.5317029953003%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.18386848270893097 - Accuracy: 93.4652566

In [17]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    model_path = "content_english/models_fasttext/" + str(fold_no) + "model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "content_english/History_fasttext/" + str(fold_no) + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
450/450 [==============================] - 207s 439ms/step - loss: 0.2345 - accuracy: 0.9295
Epoch 2/5
450/450 [==============================] - 196s 436ms/step - loss: 0.2022 - accuracy: 0.9303
Epoch 3/5
450/450 [==============================] - 196s 436ms/step - loss: 0.1931 - accuracy: 0.9334
Epoch 4/5
450/450 [==============================] - 196s 436ms/step - loss: 0.1882 - accuracy: 0.9345
Epoch 5/5
450/450 [==============================] - 196s 436ms/step - loss: 0.1919 - accuracy: 0.9333
INFO:tensorflow:Assets written to: content_english/models_fasttext/1model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/1model/assets


Score for fold 1: loss of 0.17571136355400085; accuracy of 93.78741383552551%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
450/450 [==============================] - 204s 433ms/step - loss: 0.2350 - accuracy: 0.9284
Epoch 2/5
450/450 [==============================] - 196s 435ms/step - loss: 0.2023 - accuracy: 0.9307
Epoch 3/5
450/450 [==============================] - 194s 431ms/step - loss: 0.1936 - accuracy: 0.9325
Epoch 4/5
450/450 [==============================] - 195s 433ms/step - loss: 0.1884 - accuracy: 0.9352
Epoch 5/5
450/450 [==============================] - 195s 433ms/step - loss: 0.1854 - accuracy: 0.9357
INFO:tensorflow:Assets written to: content_english/models_fasttext/2model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/2model/assets


Score for fold 2: loss of 0.19730144739151; accuracy of 93.28171014785767%
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/5
450/450 [==============================] - 207s 441ms/step - loss: 0.2339 - accuracy: 0.9286
Epoch 2/5
450/450 [==============================] - 198s 440ms/step - loss: 0.1984 - accuracy: 0.9320
Epoch 3/5
450/450 [==============================] - 198s 440ms/step - loss: 0.1945 - accuracy: 0.9327
Epoch 4/5
450/450 [==============================] - 197s 438ms/step - loss: 0.1890 - accuracy: 0.9332
Epoch 5/5
450/450 [==============================] - 199s 442ms/step - loss: 0.1842 - accuracy: 0.9357
INFO:tensorflow:Assets written to: content_english/models_fasttext/3model/assets


...

Score for fold 8: loss of 0.18081310391426086; accuracy of 93.51586699485779%
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/5
450/450 [==============================] - 205s 436ms/step - loss: 0.2322 - accuracy: 0.9301
Epoch 2/5
450/450 [==============================] - 197s 439ms/step - loss: 0.1967 - accuracy: 0.9325
Epoch 3/5
450/450 [==============================] - 195s 434ms/step - loss: 0.1921 - accuracy: 0.9332
Epoch 4/5
450/450 [==============================] - 194s 431ms/step - loss: 0.1856 - accuracy: 0.9355
Epoch 5/5
450/450 [==============================] - 194s 431ms/step - loss: 0.1842 - accuracy: 0.9366
INFO:tensorflow:Assets written to: content_english/models_fasttext/9model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/9model/assets


Score for fold 9: loss of 0.19933350384235382; accuracy of 93.04444193840027%
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/5
450/450 [==============================] - 225s 482ms/step - loss: 0.2330 - accuracy: 0.9295
Epoch 2/5
450/450 [==============================] - 214s 476ms/step - loss: 0.1998 - accuracy: 0.9318
Epoch 3/5
450/450 [==============================] - 219s 486ms/step - loss: 0.1934 - accuracy: 0.9323
Epoch 4/5
450/450 [==============================] - 210s 467ms/step - loss: 0.1878 - accuracy: 0.9348
Epoch 5/5
450/450 [==============================] - 195s 433ms/step - loss: 0.1850 - accuracy: 0.9355
INFO:tensorflow:Assets written to: content_english/models_fasttext/10model/assets


INFO:tensorflow:Assets written to: content_english/models_fasttext/10model/assets


Score for fold 10: loss of 0.17836728692054749; accuracy of 94.2636489868164%
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.17571136355400085 - Accuracy: 93.78741383552551%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.19730144739151 - Accuracy: 93.28171014785767%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.20718733966350555 - Accuracy: 93.51898431777954%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.17666363716125488 - Accuracy: 93.82510781288147%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.18341375887393951 - Accuracy: 93.60918998718262%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.18754158914089203 - Accuracy: 93.1247472